In [1]:
from haystack.preprocessor.utils import convert_files_to_dicts

12/02/2020 13:19:53 - INFO - faiss -   Loading faiss with AVX2 support.
12/02/2020 13:19:53 - INFO - faiss -   Loading faiss.


In [2]:
dicts=convert_files_to_dicts("/home/elena/Downloads/data/", split_paragraphs=True) # no cleaning function applied

12/02/2020 13:19:55 - INFO - haystack.preprocessor.utils -   Converting /home/elena/Downloads/data/9781839217579-THE_DEEP_LEARNING_WITH_KERAS_WORKSHOP_SECOND_EDITION.pdf


In [3]:
type(dicts), len(dicts), type(dicts[0]), dicts[0].keys()

(list, 1418, dict, dict_keys(['text', 'meta']))

In [4]:
print(dicts[0])

{'text': '\x0cThe Deep Learning\nwith Keras Workshop\nSecond Edition', 'meta': {'name': '9781839217579-THE_DEEP_LEARNING_WITH_KERAS_WORKSHOP_SECOND_EDITION.pdf'}}


In [3]:
from haystack.document_store.faiss import FAISSDocumentStore

document_store = FAISSDocumentStore()
document_store.delete_all_documents()

In [4]:
document_store.write_documents(dicts)

In [5]:
from haystack.retriever.dense import DensePassageRetriever
retriever = DensePassageRetriever(document_store=document_store,
                                  query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                  passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                  max_seq_len_query=64,
                                  max_seq_len_passage=256,
                                  batch_size=16,
                                  use_gpu=False,
                                  embed_title=True,
                                  use_fast_tokenizers=True)
    

12/02/2020 13:21:26 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
12/02/2020 13:21:33 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.


In [6]:
document_store.update_embeddings(retriever)

12/02/2020 13:21:38 - INFO - haystack.document_store.faiss -   Updating embeddings for 1418 docs...
Inferencing Samples: 100%|██████████| 89/89 [12:37<00:00,  8.51s/ Batches]
12/02/2020 13:34:18 - INFO - haystack.document_store.faiss -   Indexing embeddings and updating vectors_ids...
100%|██████████| 1/1 [00:00<00:00,  5.84it/s]


In [7]:
from haystack.reader.farm import FARMReader
farm_reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

12/02/2020 13:34:30 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
12/02/2020 13:34:30 - INFO - farm.infer -   Could not find `deepset/roberta-base-squad2` locally. Try to download from model hub ...
Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
12/02/2020 13:34:34 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
12/02/2020 13:34:43 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
12/02/2020 13:34:43 - INFO - farm.inf

In [8]:
from haystack import Finder

In [9]:
finder = Finder(farm_reader, retriever)

In [22]:
prediction = finder.get_answers(question="What is RNN?", top_k_retriever=10, top_k_reader=3)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  5.12 Batches/s]
12/02/2020 13:43:43 - INFO - haystack.finder -   Got 10 candidates from retriever
12/02/2020 13:43:43 - INFO - haystack.finder -   Reader is looking for detailed answer in 3458 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.52 Batches/s]


In [12]:
from haystack.utils import print_answers

In [23]:
print_answers(prediction, details="medium")

[   {   'answer': 'the hidden layer not only gives the\n'
                  'output, but it also feeds back the information of the '
                  'output into itself',
        'context': 'y of the RNN is that the hidden layer not only gives the\n'
                   'output, but it also feeds back the information of the '
                   'output into itself. Before taking a\n'
                   'dee',
        'score': 12.397671699523926},
    {   'answer': 'Recurrent Neural Networks',
        'context': 'Recurrent Neural Networks (RNNs)\n'
                   'RNNs are a class of neural networks that are built on the '
                   'concept of sequential\n'
                   'memory. Unlike traditional neural net',
        'score': 11.025992393493652},
    {   'answer': 'Long Short-Term Memory',
        'context': 'Long Short-Term Memory (LSTM)\n'
                   'LSTMs are RNNs whose main objective is to overcome the '
                   'shortcomings of the vanishin

In [24]:
prediction = finder.get_answers(question="What is a layer?", top_k_retriever=10, top_k_reader=3)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  3.93 Batches/s]
12/02/2020 13:43:58 - INFO - haystack.finder -   Got 10 candidates from retriever
12/02/2020 13:43:58 - INFO - haystack.finder -   Reader is looking for detailed answer in 5180 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  2.17 Batches/s]


In [25]:
print_answers(prediction, details="medium")

[   {   'answer': 'Dense layer',
        'context': 'ras. For now, we will use only the simplest type of\n'
                   'layer, called the Dense layer. A Dense layer is equivalent '
                   'to the fully connected\n'
                   'layers that we h',
        'score': 10.62425708770752},
    {   'answer': 'a\ncomposition of nodes',
        'context': 'ers is part of the Keras core API. A layer can be thought '
                   'of as a\n'
                   'composition of nodes, and at each node, a set of '
                   'computations happen. In Keras, all ',
        'score': 10.560615539550781},
    {   'answer': 'convolutional',
        'context': 'd: it has a four-dimensional input shape (None, 224, 224,\n'
                   '3) and it has three convolutional layers.\n'
                   'The last four layers of the output are as follows:',
        'score': 9.095014572143555}]


In [26]:
prediction = finder.get_answers(question="What is deep learning?", top_k_retriever=10, top_k_reader=3)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  4.69 Batches/s]
12/02/2020 13:44:20 - INFO - haystack.finder -   Got 10 candidates from retriever
12/02/2020 13:44:20 - INFO - haystack.finder -   Reader is looking for detailed answer in 3292 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  2.28 Batches/s]


In [27]:
print_answers(prediction, details="medium")

[   {   'answer': 'Keras',
        'context': "oss')\n"
                   "plt.xlabel('epoch')\n"
                   "plt.legend(['train loss', 'validation loss'], loc='upper "
                   "right')\n"
                   '\x0c'
                   'Chapter 3: Deep Learning with Keras | 337\n'
                   'Expected output:',
        'score': 6.224386215209961},
    {   'answer': 'cyclical: 284',
        'context': ', 254, 261, 263,\n'
                   '266, 269, 271, 274,\n'
                   '276, 294, 298, 301,\n'
                   '305-306, 308-309\n'
                   'current: 95, 113, 283,\n'
                   '287, 294, 301\n'
                   'curves: 193-194, 204, 222\n'
                   'cyclical: 284',
        'score': -4.464661598205566},
    {   'answer': 'gradient',
        'context': '237, 248, 250, 259,\n'
                   '268, 292, 298, 304\n'
                   'google: 258, 277,\n'
                   '282-283, 286, 308\n'
                   '

In [28]:
prediction = finder.get_answers(question="What is logistic regression?", top_k_retriever=10, top_k_reader=3)


Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  4.47 Batches/s]
12/02/2020 13:44:39 - INFO - haystack.finder -   Got 10 candidates from retriever
12/02/2020 13:44:39 - INFO - haystack.finder -   Reader is looking for detailed answer in 9211 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.85 Batches/s]


In [29]:
print_answers(prediction, details="medium")

[   {   'answer': 'feature\n'
                  'coefficients are learned and predictions are made by taking '
                  'the sum of the product\n'
                  'of the feature coefficients and features',
        'context': ' in which feature\n'
                   'coefficients are learned and predictions are made by '
                   'taking the sum of the product\n'
                   'of the feature coefficients and features.\n'
                   '• Decis',
        'score': 10.148681640625},
    {   'answer': 'a very simple neural network with only one hidden layer and '
                  'only\n'
                  'one node in its hidden layer',
        'context': 'logistic\n'
                   'regression involves a very simple neural network with only '
                   'one hidden layer and only\n'
                   'one node in its hidden layer.\n'
                   'An overview of the logisti',
        'score': 10.00578498840332},
    {   'answer': 'ridge

In [32]:
prediction = finder.get_answers(question="What is early stopping?", top_k_retriever=10, top_k_reader=3)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  4.62 Batches/s]
12/02/2020 13:54:21 - INFO - haystack.finder -   Got 10 candidates from retriever
12/02/2020 13:54:21 - INFO - haystack.finder -   Reader is looking for detailed answer in 6129 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  2.17 Batches/s]


In [33]:
print_answers(prediction, details="medium")

[   {   'answer': 'forcing the Keras model to stop the training when a desired '
                  'metric—for example,\n'
                  'the test error rate—is not improving anymore',
        'context': '. This\n'
                   'means forcing the Keras model to stop the training when a '
                   'desired metric—for example,\n'
                   'the test error rate—is not improving anymore. In order to',
        'score': 10.376293182373047},
    {   'answer': 'The training',
        'context': 'ovide it as a callbacks\n'
                   'argument to model.fit() and train the model. The training '
                   'will automatically stop\n'
                   'according to the EarlyStopping() callback:\n'
                   'h',
        'score': 6.640634536743164},
    {   'answer': 'the training\n'
                  'process automatically stops after about 150 epochs',
        'context': 'ing callback with patience=10 to the model, the training\n'
          

In [34]:
prediction = finder.get_answers(question="What is early stopping used for?", top_k_retriever=10, top_k_reader=3)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  4.13 Batches/s]
12/02/2020 13:54:52 - INFO - haystack.finder -   Got 10 candidates from retriever
12/02/2020 13:54:52 - INFO - haystack.finder -   Reader is looking for detailed answer in 6771 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.14 Batches/s]


In [35]:
print_answers(prediction, details="medium")

[   {   'answer': 'forcing the Keras model to stop the training when a desired '
                  'metric—for example,\n'
                  'the test error rate—is not improving anymore',
        'context': '. This\n'
                   'means forcing the Keras model to stop the training when a '
                   'desired metric—for example,\n'
                   'the test error rate—is not improving anymore. In order to',
        'score': 8.451149940490723},
    {   'answer': 'to prevent your Keras model from\n'
                  'overfitting the training data. To do this, you utilized the '
                  'EarlyStopping callback\n'
                  'and trained the model with it. We used this callback to '
                  'stop the model any time the\n'
                  'validation loss increased',
        'context': ' to prevent your Keras model from\n'
                   'overfitting the training data. To do this, you utilized '
                   'the EarlyStopping callb